# Phenolopy

## Initialise phenolopy

### Load packages

In [ ]:
%matplotlib inline
%load_ext autoreload

import os, sys
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import datacube
sys.path.append('../../../Scripts')
from dea_datahandling import load_ard
from dea_dask import create_local_dask_cluster
from dea_plotting import display_map, rgb

sys.path.append('../../modules')
import phenolopy

sys.path.append('../../shared')
import satfetcher, tools

### Set up a dask cluster

In [ ]:
# initialise the cluster. paste url into dask panel for more info
create_local_dask_cluster()

# open up a datacube connection
dc = datacube.Datacube(app='phenolopy')

## Study area and data setup

### Set study area, time range, show map

In [ ]:
# testing study area extent - yandi and roy hill
#lat_extent, lon_extent = (-22.82901, -22.67901), (118.94980, 119.29979)  # yandi
lat_extent, lon_extent = (-22.63461, -22.33461), (119.88111, 120.18111) # royhill

# display onto interacrive map
display_map(x=lon_extent, y=lat_extent)

### Load DEA ODC satellite data

In [ ]:
# provide study area name
study_area = 'royhill'

# select start and end year range
time_range = ('2016', '2020')

# set datacube query parameters
platform = 'landsat'
bands = ['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_1', 'nbart_swir_2']
#bands = ['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir_1', 'nbart_swir_2'] # sentinel
min_gooddata = 0.90

# fetch satellite data from dea ard product
ds = satfetcher.load_dea_ard(platform=platform, 
                             bands=bands, 
                             x_extent=lon_extent, 
                             y_extent=lat_extent, 
                             time_range=time_range, 
                             min_gooddata=min_gooddata, 
                             use_dask=True)

# display dataset
#ds

### Conform DEA band names

In [ ]:
# rename dea bands to common standard
ds = satfetcher.conform_dea_ard_band_names(ds=ds, 
                                           platform=platform)

# display dataset
#ds

### Calculate vegetation index

In [ ]:
# takes our dask ds and calculates veg index from spectral bands
ds = tools.calculate_indices(ds=ds, 
                             index='mavi', 
                             custom_name='veg_idx', 
                             rescale=False, 
                             drop=True)

# display dataset
#ds

## Pre-process vegetation data

### Ensure first/last date are start/end of year

In [ ]:
# append jan 1st/31st dec scenes to dataset if missing
ds = phenolopy.conform_edge_dates(ds=ds)

# display dataset
#ds

### Resample dataset into weekly medians

In [ ]:
# we need to do weekly first, then group-resample later
ds = phenolopy.resample(ds=ds, 
                        interval='1W',
                        inplace=True)

# display dataset
#ds

### Interpolate missing values

In [ ]:
# interpolate missing values
ds = phenolopy.interpolate(ds=ds, 
                           method='full', 
                           inplace=True)

# display dataset
#ds

### Group weeks into a year of weekly all-time medians

In [ ]:
# take the dataset ds and group and reduce dataset in median weeks (52 for one year)
ds = phenolopy.group(ds=ds, 
                     interval='week',
                     inplace=True)

# display dataset
#ds

In [ ]:
%autoreload
# takes our dask ds and remove outliers from data using median method
ds = phenolopy.remove_outliers(ds=ds, 
                               method='median', 
                               user_factor=2, 
                               z_pval=0.05)

# display dataset
#ds

### Resample dataset into final intervals (weekly, bi-monthly, monthly)

In [ ]:
# take dataset and resample data to bi-monthly medians (1SMS)
ds = phenolopy.resample(ds=ds, 
                        interval='1SM',
                        inplace=True)

# display dataset
#ds

### Remove first/last time if outside main year

In [ ]:
# remove any years outside of dominant year
ds = phenolopy.remove_overshoot_times(ds=ds, max_times=3)

# display dataset
#ds

## Smooth data on per-pixel basis

In [ ]:
# use savitsky-golay filter to smooth across time dimension
ds = phenolopy.smooth(ds=ds, 
                      method='savitsky', 
                      window_length=3, 
                      polyorder=1, 
                      sigma=1)

# display dataset
#ds

## Calculate Phenometrics

### Compute into memory

In [ ]:
# compute
ds = ds.compute()

# display dataset
#ds

### Calculate phenometrics

In [ ]:
%autoreload

# set desired metrics
metrics = ['sos', 'eos', 'lios', 'sios', 'liot', 'siot']

# calc phenometrics via phenolopy!
ds_phenos = phenolopy.calc_phenometrics(ds=ds, 
                                        metrics=metrics,
                                        peak_metric='pos', 
                                        base_metric='vos', 
                                        method='seasonal_amplitude', 
                                        factor=0.2, 
                                        thresh_sides='one_sided', 
                                        abs_value=0.1)

## Display phenometric

In [ ]:
# set the metric you want to view (e.g. pos_values, pos_times, vos_values, vos_times, ect.)
metric_name = 'liot_values'

# create fig
fig = plt.figure(figsize=(9, 7), dpi=85)

# plot this on map
ds_phenos[metric_name].plot(robust=True, cmap='terrain_r')

## Calculate number of seasons (optional)

In [ ]:
# take our dask ds and calculate number of seasons (num of major peaks) per-pixel
ds_nos = phenolopy.calc_num_seasons(ds=ds)

# display dataset
fig = plt.figure(figsize=(7, 7))
ds_nos['nos_values'].plot(robust=True)